In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from itertools import product

# Мапа для цільових індексів
index_map = {
    0: "AGG",
    1: "BIL",
    2: "BND",
    3: "EDV",
    4: "IEF",
    5: "IEI",
    6: "SHV",
    7: "SHY",
    8: "TLH",
    9: "TLT",
    10: "VGIT"
}

# Dataset клас
class BondDataset(Dataset):
    def __init__(self, lag, target=0, file_path="bond_etfs.csv", transform=None):
        self.lag = lag
        self.target = target
        self.records = np.genfromtxt(
            file_path,
            delimiter=",",
            skip_header=1,
            usecols=(1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12)
        ).astype(np.float32)
        self.transform = transform

    def __len__(self):
        return len(self.records) - self.lag

    def __getitem__(self, idx):
        records = self.records[idx:idx + self.lag + 1]
        x = torch.from_numpy(records[:-1])
        y = torch.from_numpy(np.atleast_1d(records[-1][self.target] - records[-2][self.target]))
        if self.transform:
            x = self.transform(x)
        return x, y


In [2]:
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



Using cpu device


In [3]:

full_dataset = BondDataset(lag=15, target=1)
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [0.8, 0.2])

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, lag):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(lag * 11, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

# LSTM модель
class LSTMModel(nn.Module):
    def __init__(self, lag, input_dim=11, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

In [12]:
from sklearn.metrics import mean_absolute_error, r2_score
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test_loop(dataloader, model, loss_fn, device):
    model.eval()
    test_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            all_preds.append(pred.cpu().numpy())
            all_labels.append(y.cpu().numpy())
    
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    
    # Обчислення метрик
    mse = np.mean((all_preds - all_labels) ** 2)
    mae = mean_absolute_error(all_labels, all_preds)
    r2 = r2_score(all_labels, all_preds)
    
    return test_loss / len(dataloader), mse, mae, r2

In [16]:
param_grid = {
    "model": ["NeuralNetwork", "LSTMModel"],
    "batch_size": [32, 64],
    "learning_rate": [1e-4, 1e-3],
    "lag": [5, 15]
}
param_combinations = list(product(*param_grid.values()))
device = "cuda" if torch.cuda.is_available() else "cpu"

# Основний цикл експериментів
results = []
for params in param_combinations:
    model_type, batch_size, learning_rate, lag = params
    print(f"Testing: Model={model_type}, BatchSize={batch_size}, LR={learning_rate}, Lag={lag}")

    # Ініціалізація датасету
    dataset = BondDataset(lag=lag, target=1)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Ініціалізація моделі
    if model_type == "NeuralNetwork":
        model = NeuralNetwork(lag).to(device)
    elif model_type == "LSTMModel":
        model = LSTMModel(lag).to(device)

    # Ініціалізація оптимізатора та функції втрат
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


Testing: Model=NeuralNetwork, BatchSize=32, LR=0.0001, Lag=5
Testing: Model=NeuralNetwork, BatchSize=32, LR=0.0001, Lag=15
Testing: Model=NeuralNetwork, BatchSize=32, LR=0.001, Lag=5
Testing: Model=NeuralNetwork, BatchSize=32, LR=0.001, Lag=15
Testing: Model=NeuralNetwork, BatchSize=64, LR=0.0001, Lag=5
Testing: Model=NeuralNetwork, BatchSize=64, LR=0.0001, Lag=15
Testing: Model=NeuralNetwork, BatchSize=64, LR=0.001, Lag=5
Testing: Model=NeuralNetwork, BatchSize=64, LR=0.001, Lag=15
Testing: Model=LSTMModel, BatchSize=32, LR=0.0001, Lag=5
Testing: Model=LSTMModel, BatchSize=32, LR=0.0001, Lag=15
Testing: Model=LSTMModel, BatchSize=32, LR=0.001, Lag=5
Testing: Model=LSTMModel, BatchSize=32, LR=0.001, Lag=15
Testing: Model=LSTMModel, BatchSize=64, LR=0.0001, Lag=5
Testing: Model=LSTMModel, BatchSize=64, LR=0.0001, Lag=15
Testing: Model=LSTMModel, BatchSize=64, LR=0.001, Lag=5
Testing: Model=LSTMModel, BatchSize=64, LR=0.001, Lag=15


In [18]:


 for epoch in range(5):  # Кількість епох можна змінити
        train_loop(train_loader, model, loss_fn, optimizer, device)

    # Тестування
        test_loss, mse, mae, r2 = test_loop(test_loader, model, loss_fn, device)

      # Виведення результатів кожної епохи
        print(f"Epoch {epoch+1} Results: ")
        print(f"Test Loss: {test_loss:.4f}")
        print(f"MSE: {mse:.4f}")
        print(f"MAE: {mae:.4f}")
        print(f"R²: {r2:.4f}")
        print("-" * 40)

    # Збереження результатів
        results.append({
            "Model": model_type,
            "BatchSize": batch_size,
            "LearningRate": learning_rate,
            "Lag": lag,
            "TestLoss": test_loss,
            "MSE": mse,
            "MAE": mae,
            "R2": r2     
    })

# Збереження результатів у CSV
df = pd.DataFrame(results)
df.to_csv("experiment_results.csv", index=False)
print("Results saved to 'experiment_results.csv'")

Epoch 1 Results: 
Test Loss: 0.0028
MSE: 0.0027
MAE: 0.0213
R²: -0.0593
----------------------------------------
Epoch 2 Results: 
Test Loss: 0.0027
MSE: 0.0027
MAE: 0.0196
R²: -0.0386
----------------------------------------
Epoch 3 Results: 
Test Loss: 0.0027
MSE: 0.0026
MAE: 0.0191
R²: -0.0063
----------------------------------------
Epoch 4 Results: 
Test Loss: 0.0027
MSE: 0.0026
MAE: 0.0214
R²: -0.0056
----------------------------------------
Epoch 5 Results: 
Test Loss: 0.0028
MSE: 0.0027
MAE: 0.0268
R²: -0.0524
----------------------------------------
Results saved to 'experiment_results.csv'


In [19]:
X, Y = next(iter(test_loader))  # Замість test_dataloader
print(X[0])
print(Y[0])


tensor([[100.7100,  91.5500,  74.6000,  96.5600, 100.9400, 117.8900, 110.1300,
          82.2900, 117.6100, 111.8800,  60.4000],
        [100.0700,  91.4300,  74.0400,  94.3400,  99.9600, 117.3300, 109.9700,
          82.1300, 115.6300, 109.6000,  60.0300],
        [100.3600,  91.4400,  74.2300,  94.8400, 100.5200, 117.8600, 109.9700,
          82.3100, 116.2600, 110.2200,  60.3300],
        [ 99.4200,  91.4400,  73.5600,  91.7100,  99.4000, 117.1500, 109.9800,
          82.1300, 114.0300, 107.4900,  59.8600],
        [100.1100,  91.4500,  74.0900,  93.6600, 100.0500, 117.6400, 109.9800,
          82.2500, 115.4500, 109.1900,  60.1500],
        [ 99.7900,  91.4700,  73.8600,  92.4600,  99.6800, 117.3300, 109.9800,
          82.1500, 114.8000, 108.0700,  59.9800],
        [ 99.7800,  91.4700,  73.8300,  92.5500,  99.6900, 117.2500, 109.9800,
          82.0800, 114.7100, 108.3100,  59.9300],
        [ 99.6300,  91.4600,  73.7400,  91.5500,  99.4500, 117.1900, 109.9800,
          82.0700,